In [4]:
import dask.dataframe as dd

# Read a large CSV file
df = dd.read_csv('/kaggle/input/leash-BELKA/train.csv')

# View a few rows
df.head()


,id,buildingblock1_smiles,buildingblock2_smiles,buildingblock3_smiles,molecule_smiles,protein_name,binds
0,0,C#CC[C@@H](CC(=O)O)NC(=O)OCC1c2ccccc2-c2ccccc21,C#CCOc1ccc(CN)cc1.Cl,Br.Br.NCC1CCCN1c1cccnn1,C#CCOc1ccc(CNc2nc(NCC3CCCN3c3cccnn3)nc(N[C@@H]...,BRD4,0
1,1,C#CC[C@@H](CC(=O)O)NC(=O)OCC1c2ccccc2-c2ccccc21,C#CCOc1ccc(CN)cc1.Cl,Br.Br.NCC1CCCN1c1cccnn1,C#CCOc1ccc(CNc2nc(NCC3CCCN3c3cccnn3)nc(N[C@@H]...,HSA,0
2,2,C#CC[C@@H](CC(=O)O)NC(=O)OCC1c2ccccc2-c2ccccc21,C#CCOc1ccc(CN)cc1.Cl,Br.Br.NCC1CCCN1c1cccnn1,C#CCOc1ccc(CNc2nc(NCC3CCCN3c3cccnn3)nc(N[C@@H]...,sEH,0
3,3,C#CC[C@@H](CC(=O)O)NC(=O)OCC1c2ccccc2-c2ccccc21,C#CCOc1ccc(CN)cc1.Cl,Br.NCc1cccc(Br)n1,C#CCOc1ccc(CNc2nc(NCc3cccc(Br)n3)nc(N[C@@H](CC...,BRD4,0
4,4,C#CC[C@@H](CC(=O)O)NC(=O)OCC1c2ccccc2-c2ccccc21,C#CCOc1ccc(CN)cc1.Cl,Br.NCc1cccc(Br)n1,C#CCOc1ccc(CNc2nc(NCc3cccc(Br)n3)nc(N[C@@H](CC...,HSA,0


In [3]:
df['binds'].value_counts().compute()

binds
0    293656924
1      1589906
Name: count, dtype: int64

In [3]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.9 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812364 sha256=5c01262d4ca5ceff1c787e50317be6de4af8b89cc66eaa614e6e253829afa4a4
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


In [4]:
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder.appName("merge_example").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/21 15:27:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
spark

In [6]:


# Load the two datasets
df1 = spark.read.csv('/kaggle/input/class-to-nd/Clas_to_nd.csv', header=True, inferSchema=True)
df2 = spark.read.parquet('/kaggle/input/vol-to-oveall/vol_to_overall_average.parquet')

# Perform an outer join on the 'shopCode' column
merged_df = df1.join(df2, on='shopCode', how='outer')



In [7]:
merged_df.show()

24/09/21 15:28:58 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------+-----+---------+--------+-----------+----+------+----------+---------------+-----------------+----------------+-------------+----+-------+------+--------------------+-------------+-----------+---------------------+-------------------------------------+----------------------------+--------------------+-------------------------+-------------------------+--------------------+------------------------+---------------------+------------------+-----------------+---------------+--------+-----------+------------------+---------------+-------------------------------+--------+----+--------------------+
|shopCode|  SKU|ND_values|   Brand|Brand_value|Type| Visit|      City|Recruitment Day|Recruitment Month|Recruitment Year|         Week|Area|SubArea|Street|              Market|ContactPerson|  ContactNo|R1-Channel Typer (SR)|R2-Wholesaler Activity for CSD (SR) A|R3-Exclusivity Status (SR) A|R4-Service Type (SR)|R7-Store Location--A (SR)|R8-Store Location--B (SR)|R9a1-Food Categories|R9a2-N